## EASE

### Q13

Like in many recommender papers that use implicit feedback data, we assume that the
data are given in terms of a sparse (typically binary) matrix $X \in R^{|U|\times|I|}$, regarding the
sets of users U and items I, where $| · |$ denotes the size of a set. A positive value (typically 1) in X indicates that the user interacted with an item, while a value of 0 indicates that
no interaction has been observed. The parameters of the EASE model are given by the
item-item weight-matrix $B \in R^{|I|\times|I|}$. Note that this is also similar to neighborhood-based approaches.

In this weight matrix, self-similarity of an item in the input-layer with itself in the output layer is forbidden as to force the model to generalize when reproducing the input as its output: hence the diagonal of the weight-matrix is constrained to zero, $diag(B) = 0$.
This constraint is crucial. The predicted score $S_{uj}$ for an item $j \in I$ given a user $u \in U$ is
defined by the dot product $S_{uj} = X_{u,:} B_{:,j}$,
where $X_{u,:}$ refers to row $u$, and $B_{:,j}$ $j$ to column $j$.

Therefore each row in the matrix $S$ corresponds to a user. The similarity
scores for a given movie in the matrix B (corresponding to a row of B) are multiplied by the score given to that movie by the user, and this is done for all movies. The resulting rows are then
summed together.

### Q14

The recommendation score on an un-purchased/rated item $t_j$ of a user $u_i$ is calculated as a sparse aggregation of items that have been purchased/rated by $u_i$, that is,

(Equation $1$)
$$\tilde{x}_{ij} = x^T_i b_j$$

where $x_{ij} = 0$ and $w_j$ is a sparse size $n$ column vector of aggregation coefficients. Thus, the model utilized by SLIM can be presented as:

(Equation $2$)
$$\tilde{S} = XB$$

where X is the binary user-item purchase matrix or the user-item rating matrix, $B$ is an $n \times n$ sparse matrix of aggregation coefficients, whose $j$-th column corresponds to $b_j$
as in Equation $1$, and each row $\tilde{x}^T = x^T_iW$ of $\tilde{X}$

represents the recommendation
scores on all items for user ui. Top $N$ recommendation for $u_i$ is done by sorting $u_i$’s
nonpurchased/rated items based on their recommendation scores in $\tilde{a}^⊤_i$ in decreasing order and recommending the top- N items.

### Q15

The constraint $diag(\hat{B})=0$ is applied so as to avoid trivial solutions (the optimal $B$ is an identical matrix such that an item always recommends itself so as to minimize minB $||X − X||^2_F$.In addition, the constraint $diag(\hat{B})=0$ ensures that xij is not used to compute
$\hat{x}_{ij}$).

### Q16

In this section, we show that the constrained convex optimization problem for learning the weight matrix B can be solved in closed form:

$$ min_B ||X-XB||^2_F + \lambda||B||^2_F $$
$$ s.t. \quad diag(B)=0 $$

We start by including the equality constraint into the objective function in by forming the Lagrangian:

$$ L=||X_XB||^2_F + \lambda||B||^2_F+2\gamma^T.diag(B) $$

where $\gamma =
(γ1,... ,\gamma_{|I|})^⊤$ is the vector of Lagrangian multipliers. Its values will be chosen
later such that the constraint in convex optimization problem is fulfilled.

The constrained optimization problem is solved by minimizing this Lagrangian. As a
necessary condition, we hence set its derivative to zero, which yields the estimate $\hat{B} of the
weight matrix after re-arranging terms:

$$\hat{B} = (X^TX + \lambda I)^{-1}(X^TX-diagMat(\gamma))$$

where diagMat (·) denotes the diagonal matrix and $I$ the identity matrix. Defining (for sufficiently large $\gamma$)

$$ \hat{P} \triangleq (X^TX + \lambda I)^{-1} $$

this can be substituted into the previous equation:

$$\hat{B} = (X^TX + \lambda I)^{-1}(X^TX-diagMat(\gamma))$$
$$ = \hat{P} (\hat{P}^{-1} - \lambda I - diagMat(\gamma)) $$
$$ = I-\hat{P}(\lambda I + diagMat(\gamma))$$
$$ I-\hat{P}.diagMat(\tilde{\gamma}) $$
(Equ. 5)

where we defined the vector $\tilde{\gamma} \triangleq \gamma \overrightarrow{1} + \gamma$ in the last line, with
$\overrightarrow{1}$ denoting a vector of ones.
The values of the Lagrangian multipliers $\gamma$, and hence $\tilde{γ}$, are determined by the constraint
$diag(\tilde{B})=0$.

$$ 0 = diag(\hat{B} = \overrightarrow{1}-diag(\hat{P}) \odot \tilde{\gamma} $$

where $\odot$ denotes the elementwise product, and hence:

$$ \tilde{\gamma} = \overrightarrow{1} ⊘ diag(\hat{P}) $$

where ⊘ denotes the elementwise division (which is well-defined given that $\hat{P}$ is invertible).
Substituting this into Eq. 5 immediately results in the closed-form solution:

$$ \hat{B} = I-\hat{P}.diagMat(\overrightarrow{1} ⊘ diag(\hat{P}))$$

In other words, the learned weights are given by:

$$ \hat{B}_{i,j} = \left\{ \begin{array}{rcl}
0 & \mbox{for} & \mbox{if}\quad i=j \\
-\frac{\hat{P}_{ij}}{\hat{P}_{jj}} & \mbox{for} & \mbox{otherwize}
\end{array}\right. $$

This solution obviously obeys the constraint of a zero diagonal.

### Q17

In [1]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

!pip install opendatasets
import opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# username = 'alinourian'
# key = '4200c3f083710c14f67a2dab913f61e6'

opendatasets.download("https://www.kaggle.com/datasets/odedgolden/movielens-1m-dataset/u.data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alinourian
Your Kaggle Key: ··········


100%|██████████| 5.83M/5.83M [00:01<00:00, 4.52MB/s]


In [3]:
users_col = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('/content/movielens-1m-dataset/users.dat', sep='::', header=None, names=users_col, engine='python', encoding='latin-1')

movies_col = ['movie_id', 'title', 'genres']
movies = pd.read_table('/content/movielens-1m-dataset/movies.dat', sep='::', header=None, names=movies_col, engine='python', encoding='latin-1')

ratings_col = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('/content/movielens-1m-dataset/ratings.dat', sep='::', header=None, names=ratings_col, engine='python', encoding='latin-1')

In [4]:
ratings.drop("timestamp", inplace=True, axis=1) 

user_ids = ratings['user_id'].unique()

for user in user_ids:
    temp = ratings[ratings['user_id'] == user]
    if(temp.shape[0] < 20):
        ratings.drop(ratings.index[ratings['user_id'] == user], inplace=True)

### Q18

In [5]:
df = ratings #.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

train_df, test_df = train_test_split(df, test_size=0.2)

### Q19

In [6]:
def prepare_data(ratings, users, movies):
    len_user_total , len_movie_total = users['user_id'].max(), movies['movie_id'].max()
    X = np.zeros((len_user_total, len_movie_total))
    for index, row in tqdm(users.iterrows()):
        user_id = row['user_id']
        rateing_of_user = ratings.loc[ratings['user_id'] == user_id]
        for index2, row2 in rateing_of_user.iterrows():
            X[user_id-1 , row2['movie_id']-1] = row2['rating']
    return X

def train(X, lambda_):
    G = X.T.dot(X) + lambda_ * np.identity(X.shape[1])
    P = np.linalg.inv(G)
    B = P / (-np.diag(P))
    B += - np.diag(np.diag(B))
    return B

def test(X, B):
    return X.dot(B)

In [7]:
df = df.sample(frac=1)
data = prepare_data(df, users, movies)

0it [00:00, ?it/s]

#### Cross Validation

In [17]:
lambdas = [0.01, 0.1, 0.5, 2, 5, 10, 20]

n = int(0.2 * data.shape[0])

for lambda_ in lambdas:
    score = 0
    for i in tqdm(range(5)):
        X_train = np.vstack((data[:i*n], data[(i+1)*n:]))
        X_test = data[i*n:(i+1)*n]

        B = train(X_train, lambda_)
        Y = test(X_test, B)
        score += r2_score(X_test.flatten(), Y.flatten())
    score /= 5
    print(f'lambda = {lambda_}\t=>\tscore = {score}')

  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 0.01	=>	score = -4.688614498695406


  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 0.1	=>	score = -3.1506499949850504


  0%|          | 0/5 [00:00<?, ?it/s]

...

lambda = 5	=>	score = -0.8674263432013305


  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 10	=>	score = -0.5828032153459036


  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 20	=>	score = -0.3437526816984371


In [18]:
lambdas = [20, 50, 100, 200, 500]

n = int(0.2 * data.shape[0])

for lambda_ in lambdas:
    score = 0
    for i in tqdm(range(5)):
        X_train = np.vstack((data[:i*n], data[(i+1)*n:]))
        X_test = data[i*n:(i+1)*n]

        B = train(X_train, lambda_)
        Y = test(X_test, B)
        score += r2_score(X_test.flatten(), Y.flatten())
    score /= 5
    print(f'lambda = {lambda_}\t=>\tscore = {score}')

  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 20	=>	score = -0.3437526816984371


  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 50	=>	score = -0.09367791619737882


  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 100	=>	score = 0.05191752100528986


  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 200	=>	score = 0.1663634231142631


  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 500	=>	score = 0.27864861040422684


In [19]:
lambdas = [1000, 2000, 5000, 10000]

n = int(0.2 * data.shape[0])

for lambda_ in lambdas:
    score = 0
    for i in tqdm(range(5)):
        X_train = np.vstack((data[:i*n], data[(i+1)*n:]))
        X_test = data[i*n:(i+1)*n]

        B = train(X_train, lambda_)
        Y = test(X_test, B)
        score += r2_score(X_test.flatten(), Y.flatten())
    score /= 5
    print(f'lambda = {lambda_}\t=>\tscore = {score}')

  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 1000	=>	score = 0.3391452236513575


  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 2000	=>	score = 0.38152144580637937


  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 5000	=>	score = 0.41256763454532025


  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 10000	=>	score = 0.4190844554718369


In [20]:
lambdas = [20000, 50000]

n = int(0.2 * data.shape[0])

for lambda_ in lambdas:
    score = 0
    for i in tqdm(range(5)):
        X_train = np.vstack((data[:i*n], data[(i+1)*n:]))
        X_test = data[i*n:(i+1)*n]

        B = train(X_train, lambda_)
        Y = test(X_test, B)
        score += r2_score(X_test.flatten(), Y.flatten())
    score /= 5
    print(f'lambda = {lambda_}\t=>\tscore = {score}')

  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 20000	=>	score = 0.41321260497337275


  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 50000	=>	score = 0.3906409787066193


In [21]:
lambdas = [7500, 15000]

n = int(0.2 * data.shape[0])

for lambda_ in lambdas:
    score = 0
    for i in tqdm(range(5)):
        X_train = np.vstack((data[:i*n], data[(i+1)*n:]))
        X_test = data[i*n:(i+1)*n]

        B = train(X_train, lambda_)
        Y = test(X_test, B)
        score += r2_score(X_test.flatten(), Y.flatten())
    score /= 5
    print(f'lambda = {lambda_}\t=>\tscore = {score}')

  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 7500	=>	score = 0.4180168508292882


  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 15000	=>	score = 0.41699609632258416


### Q20

In [22]:
lambda_ = 10000

n = int(0.2 * data.shape[0])

score = 0
for i in tqdm(range(5)):
    X_train = np.vstack((data[:i*n], data[(i+1)*n:]))
    X_test = data[i*n:(i+1)*n]

    B = train(X_train, lambda_)
    Y = test(X_test, B)
    score += r2_score(X_test.flatten(), Y.flatten())
score /= 5
print(f'lambda = {lambda_}\t=>\tscore = {score}')

  0%|          | 0/5 [00:00<?, ?it/s]

lambda = 10000	=>	score = 0.4190844554718369
